In [1]:
import os
import sys
folder = os.path.dirname('../../../algorithms/')
sys.path.append(folder)

from PG.trainer import PG_Trainer

In [2]:
env_name = 'CartPole-v0'
lr_base = 0.001
lr_decay = 0.0001

random_seed = 42
gamma = 0.99                # discount for future rewards
batch_size = 32         # num of transitions sampled from replay buffer

max_episodes = 100000         # max num of episodes
max_timesteps = 3000        # max timesteps in one episode
log_interval = 50           # print avg reward after interval

entropy_beta = 0.01
bellman_steps = 10
baseline_steps = 50000


In [3]:
config = [
     {'dim': [None, 128], 'dropout': False, 'activation': 'relu'},    
     {'dim': [128, None], 'dropout': False, 'activation': False},    
] 

In [4]:
agent = PG_Trainer(env_name, config, random_seed=random_seed, lr_base=lr_base, lr_decay=lr_decay, 
                   gamma=gamma, batch_size=batch_size,
                   max_episodes=max_episodes, max_timesteps=max_timesteps, 
                   log_interval=log_interval, entropy_beta=entropy_beta, 
                   bellman_steps=bellman_steps, baseline_steps=baseline_steps
                   )
agent.train()

NETWORK: Sequential(
  (0): Linear(in_features=4, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=2, bias=True)
) Device: cuda:0
Random Seed: 42
action_space=Discrete(2)
obs_space=Box(4,)
threshold=195.0 

DIR=./preTrained/pg NAME=pg_CartPole-v0_42
No models to load

Training started ... 
Ep:   50  Rew:   32.00  Avg Rew:   29.32  LR:0.00099512  Loss: 0.024668
Ep:  100  Rew:   16.00  Avg Rew:   35.45  LR:0.00099020  Loss: 0.131598
Ep:  150  Rew:   42.00  Avg Rew:   46.32  LR:0.00098532  Loss: 0.119714
Ep:  200  Rew:  106.00  Avg Rew:   77.91  LR:0.00098049  Loss: 0.260722
Ep:  250  Rew:  200.00  Avg Rew:  133.91  LR:0.00097570  Loss: 0.173722
Ep:  300  Rew:  200.00  Avg Rew:  155.66  LR:0.00097097  Loss: 0.097179
Ep:  350  Rew:  200.00  Avg Rew:  165.20  LR:0.00096628  Loss: 0.091477
Ep:  400  Rew:  200.00  Avg Rew:  187.95  LR:0.00096163  Loss: 0.083099
Ep:  449  Rew:  200.00  Avg Rew:  195.24  LR:0.00095703  Loss: 0.040412
########## Solved! ####

In [5]:
agent.test()

Test episode: 1	Reward: 200.00
Test episode: 2	Reward: 200.00
Test episode: 3	Reward: 200.00
